In [1]:
import os
import json
import re
import pandas as pd
import googlemaps

In [2]:
%run ../config.py #load proj configs

In [3]:
def dados_banco_dados():
    
    path_dados = '../Dados_originais/Dados_Banco_de_Dados'
    files_site = [os.path.join(path_dados, file) for file in os.listdir(path_dados)]
    
    contrib_f = [f for f in files_site if f.split('\\')[-1].startswith('contrib')][0]
    prop_f = [f for f in files_site if f.split('\\')[-1].startswith('result')][0]
    
    return pd.read_csv(contrib_f), pd.read_csv(prop_f)

In [4]:
def pegar_enderecos(contrib, prop):
    
    enderecos = pd.concat([prop['Endereco'],contrib['Endereço']])
    
    return enderecos.unique()

In [5]:
def geocode(gmaps_client, end):
    
    geocode_result = gmaps_client.geocode(end)
    
    return geocode_result

In [6]:
def salvar_json_end(dados, version, path_salvar = None):
    
    if path_salvar is None:
        path_salvar = '../Dados_gerados/geocode'
        if not os.path.exists(path_salvar):
            os.mkdir(path_salvar)
    
    fname = os.path.join(path_salvar, f'enderecos_georref_{version}.json')
    
    with open(fname, 'w') as f:
        json.dump(dados, f)
    

In [7]:
def get_end_salvo(versao, path = None):
    
    if path is None:
        path = '../Dados_gerados/geocode'
    if os.path.exists(path):
        pat = 'enderecos_georref_\d*.json'
        files = [os.path.join(path, file) for file in os.listdir(path)
                if re.match(pat, file)]

        for f in files:
            versao_file = os.path.split(f)[-1].split('_')[-1].split('.')[0]
            if str(versao) == versao_file:
                with open(f) as file:
                    return json.load(file)
    
    
    return None

In [8]:
def pegar_todos_end(gmaps_client, enderecos, versao, salvar = True, path = None):
    
    versao_existente = get_end_salvo(versao, path)
    
    if versao_existente:
        print(f'Retornando versão {versao} já salva')
        return versao_existente
    
    
    end_geocode = {}
    count_requisi = 0
    total_requisi = len(enderecos)
    for end in enderecos:
        end_geocode[end] = geocode(gmaps_client, end)
        count_requisi+=1
        if not count_requisi%10:
            print(f'{count_requisi} requisiçoes feitas.')
            print(f'{round(count_requisi/total_requisi, 2)*100}% do total')
    if salvar:
        salvar_json_end(end_geocode, versao, path)
    
    return end_geocode
    

In [9]:
contrib, prop = dados_banco_dados()

In [10]:
enderecos = pegar_enderecos(contrib, prop)

In [11]:
gmaps_client = googlemaps.Client(key=google_api_key)

In [12]:
ends_georref = pegar_todos_end(gmaps_client, enderecos, versao = 1)

Retornando versão 1 já salva


In [13]:
ends_georref

{'Rua Cachoeira Camaleão 247  São Paulo SP': [{'address_components': [{'long_name': '247',
     'short_name': '247',
     'types': ['street_number']},
    {'long_name': 'Rua Cachoeira Camaleão',
     'short_name': 'R. Cachoeira Camaleão',
     'types': ['route']},
    {'long_name': 'Conjunto Habitacional Inacio Monteiro',
     'short_name': 'Conj. Hab. Inacio Monteiro',
     'types': ['political', 'sublocality', 'sublocality_level_1']},
    {'long_name': 'São Paulo',
     'short_name': 'São Paulo',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'São Paulo',
     'short_name': 'SP',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Brazil',
     'short_name': 'BR',
     'types': ['country', 'political']},
    {'long_name': '08472-150',
     'short_name': '08472-150',
     'types': ['postal_code']}],
   'formatted_address': 'R. Cachoeira Camaleão, 247 - Conj. Hab. Inacio Monteiro, São Paulo - SP, 08472-150, Brazil',
   'geomet